# 03 • Model & Blending (CV → OOF → Calib/τ → Blend → Test)

Эта тетрадка отвечает за обучение и блендинг моделей на подготовленных фичах.
Она берёт данные из `artifacts/sets/<run_tag>/…`, тренирует 1–3 кандидата,
делает калибровку/порог, блендинг и выдаёт итоговые предсказания.

Артефакты моделей кладутся в `artifacts/models/<run_id>`; ничего не скрывается.
Все обучаемые трансформы, калибровки и пороги считаются **только по OOF** (anti-leak).

Минимальный план: один GBDT + одна линейка. Усиление: добавить бленд + калибровку/τ.


In [ ]:
import os, sys, json, time, gc, math, warnings, hashlib
import numpy as np
import pandas as pd
from pathlib import Path

BASE = os.getenv("BASE", ".")
sys.path.append(str(Path(BASE).resolve()))

try:
    from tqdm.auto import tqdm
except Exception:
    def tqdm(x, **kw): return x

# наши модули
from common.io import load_set
from common.models import gbdt, linear, blend, calibration, thresholds, eval as ME, artifacts as MA
from common.features import assemble  # для валидации пар
warnings.filterwarnings("ignore")

def short_hash(s: str) -> str:
    return hashlib.sha1(s.encode()).hexdigest()[:8]

def mem_gb(obj) -> float:
    if hasattr(obj, "memory_usage"):
        try: return float(obj.memory_usage(deep=True).sum())/(1024**3)
        except: pass
    try: return float(np.array(obj).nbytes)/(1024**3)
    except: return 0.0

print("Python:", sys.version)



## Источники фич: из сохранённого набора

* Читаем всё из `artifacts/sets/<run_tag>`: dense/sparse, таргет, фолды, метаданные.
* Если `FOLDS` на диске нет — fallback ниже построит KFold.
* Каталог фич подтягиваем из `meta.json` (если нужен для анализа).

In [ ]:
# ——— откуда брать фичи
RUN_TAG    = "exp01"   # набор фичей из artifacts/sets/<RUN_TAG>

# ——— базовые столбцы (для метаданных/разрезов)
ID_COL     = "id"
TARGET_COL = None      # заполнить, если есть
DATE_COL   = None
GEO_COLS   = []        # например ["lat_bin_1000"]

# ——— тип задачи и метрика
TASK_TYPE      = "binary"      # regression | binary | multiclass | multilabel
PRIMARY_METRIC = "roc_auc"     # см. get_scorer в common.models.eval
TOP_K          = None          # для multilabel@k

# ——— флаги
FAST = True   # ускоренные профили кандидатов
SAFE = True   # строгие анти-утечки/фиксация сидов

# ——— управление кандидатами
USE_GBDT_DENSE    = True
USE_LINEAR_SPARSE = True
USE_HYBRID        = False   # например, отдельный GBDT на поднаборе столбцов (если нужно)

# ——— настройка GBDT/Linear по умолчанию (можно менять ниже)
GBDT_PARAMS = dict(n_estimators=400 if FAST else 800, learning_rate=0.05, max_depth=8, subsample=0.9)
GBDT_LIB    = "lightgbm"  # "lightgbm"|"catboost"|"xgboost"
LIN_ALGO    = "lr"        # "lr"|"sgd"|"ridge"|"lasso"
LIN_PARAMS  = dict(C=2.0, max_iter=200, n_jobs=8)

SEED   = 42
N_JOBS = 8



In [ ]:
X_dense_tr, X_dense_te, X_sparse_tr, X_sparse_te, y, FOLDS, meta = load_set(RUN_TAG)

feature_catalog = meta.get("catalog", {})
ID_COL     = ID_COL or meta.get("id_col", "id")
TARGET_COL = TARGET_COL or meta.get("target_col", TARGET_COL)

if X_dense_tr is not None and X_dense_te is not None:
    assemble._validate_pair(X_dense_tr, X_dense_te)
if X_sparse_tr is not None and X_sparse_te is not None:
    assert X_sparse_tr.shape[1] == X_sparse_te.shape[1], "sparse dims mismatch"

print("Dense:", None if X_dense_tr is None else X_dense_tr.shape,
      "| Sparse:", None if X_sparse_tr is None else X_sparse_tr.shape)
print("Folds:", len(FOLDS))



## Фолды: что считаем «правильным»

* Лучше использовать `FOLDS` из слоя фич (Time/Group/KFold), чтобы согласовать OOF.
* Если нет — ниже fallback на `KFold`.
* Перед анализом полезно посмотреть размеры валидаций по фолдам.

In [ ]:
from sklearn.model_selection import KFold
if not FOLDS:
    print("[info] FOLDS не переданы — строим KFold fallback")
    idx = np.arange(X_dense_tr.shape[0] if X_dense_tr is not None else X_sparse_tr.shape[0])
    kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
    FOLDS = [(tr, va) for tr, va in kf.split(idx)]
print("Folds:", len(FOLDS), "| fold sizes:", [len(va) for _,va in FOLDS][:10])



## Кандидаты: когда какой

* **GBDT на DENSE** — табличка/гео/img: быстрый и сильный.
* **Линейка на SPARSE** — TF-IDF/char: дешёвый и надёжный.
* **Гибрид** — по желанию (поднабор столбцов или иные гиперы).

In [ ]:
CANDIDATES = []

def register_candidate(name: str, run):
    CANDIDATES.append({"name": name, "run": run})
    print(f"[ok] {name}: CV {run.cv_mean:.5f} ± {run.cv_std:.5f} → {run.artifacts_path}")


In [ ]:
if USE_GBDT_DENSE and (X_dense_tr is not None):
    try:
        feat_hash = short_hash("dense:" + ",".join(map(str, X_dense_tr.columns[:50]))) if hasattr(X_dense_tr, "columns") else short_hash("dense:"+str(X_dense_tr.shape[1]))
        run_id = MA.make_run_id(task=TASK_TYPE, model=f"{GBDT_LIB}", feat_hash=feat_hash, seed=SEED)
        run = gbdt.train_cv(
            X_dense_tr, y, X_dense_te, FOLDS,
            params=GBDT_PARAMS, run_id=run_id, lib=GBDT_LIB,
            task=TASK_TYPE, seed=SEED, n_jobs=N_JOBS, save=True, resume=True, show_progress=True, verbose=True
        )
        register_candidate("gbdt_dense", run)
    except Exception as e:
        print("gbdt_dense error:", e)
else:
    print("gbdt_dense: пропущен")


In [ ]:
if USE_LINEAR_SPARSE and (X_sparse_tr is not None):
    try:
        feat_hash = short_hash(f"sparse:{X_sparse_tr.shape[1]}")
        run_id = MA.make_run_id(task=TASK_TYPE, model=f"{LIN_ALGO}", feat_hash=feat_hash, seed=SEED)
        run = linear.train_cv(
            X_sparse_tr, y, X_sparse_te, FOLDS,
            algo=LIN_ALGO, task=TASK_TYPE, params=LIN_PARAMS,
            run_id=run_id, seed=SEED, n_jobs=N_JOBS, save=True, resume=True, show_progress=True, verbose=True
        )
        register_candidate("linear_sparse", run)
    except Exception as e:
        print("linear_sparse error:", e)
else:
    print("linear_sparse: пропущен")


## (Опционально) Кандидат #3: гибрид

Например, обучить ещё один GBDT на поднаборе столбцов или с более консервативными параметрами.

In [ ]:
if USE_HYBRID and (X_dense_tr is not None):
    try:
        params = {**GBDT_PARAMS, "max_depth": 6}
        feat_hash = short_hash("dense_hybrid:" + (",".join(map(str, X_dense_tr.columns[:50])) if hasattr(X_dense_tr, "columns") else str(X_dense_tr.shape[1])))
        run_id = MA.make_run_id(task=TASK_TYPE, model=f"{GBDT_LIB}-hyb", feat_hash=feat_hash, seed=SEED)
        run = gbdt.train_cv(
            X_dense_tr, y, X_dense_te, FOLDS,
            params=params, run_id=run_id, lib=GBDT_LIB,
            task=TASK_TYPE, seed=SEED, n_jobs=N_JOBS, save=True, resume=True, show_progress=True, verbose=True
        )
        register_candidate("gbdt_hybrid", run)
    except Exception as e:
        print("hybrid error:", e)
else:
    print("hybrid: пропущен")


## Сравнение кандидатов: таблица и корреляции

Сводка CV (mean/std), ссылки на артефакты и корреляции OOF помогают перед блендингом.

In [ ]:
def collect_oof_matrix(cands):
    mats = []
    names = []
    for c in cands:
        r = c["run"]
        arr = r.oof_pred
        if arr.ndim>1 and arr.shape[1]>1:
            arr = arr.max(1)
        mats.append(arr.reshape(-1,1))
        names.append(c["name"])
    return np.hstack(mats), names

rows = []
for c in CANDIDATES:
    r = c["run"]
    rows.append(dict(name=c["name"], cv_mean=r.cv_mean, cv_std=r.cv_std, path=str(r.artifacts_path)))
cv_df = pd.DataFrame(rows).sort_values("cv_mean", ascending=False) if rows else pd.DataFrame()
display(cv_df)

try:
    if rows:
        oof_mat, names = collect_oof_matrix(CANDIDATES)
        corr = pd.DataFrame(np.corrcoef(oof_mat.T), index=names, columns=names)
        display(corr)
except Exception as e:
    print("corr skipped:", e)


## Быстрый HPO (микро-рандом)

* 10–25 запусков по ключевым гиперпараметрам GBDT (`num_leaves/max_depth/learning_rate`).
* Ограничить время на фолд; остановиться, если нет прироста за 10–15 минут.

In [ ]:
def mini_hpo_gbdt(n_trials=5):
    """Пример оболочки для микро-HPO; заполни по необходимости."""
    trials = []
    for i in range(n_trials):
        trial_params = {**GBDT_PARAMS}
        trial_params["max_depth"] = np.random.choice([6,8,10])
        trial_params["learning_rate"] = float(np.random.choice([0.03,0.05,0.1]))
        print(f"[hpo] trial {i+1}/{n_trials}: {trial_params}")
        # вставь запуск train_cv(...) по желанию
        trials.append({"params": trial_params, "cv_mean": None})
    return trials


## Блендинг: equal-weight / weight-search / level-2

* Equal-weight — быстро и часто достаточно.
* Weight-search — ищем веса по OOF под целевую метрику.
* Level-2 (Ridge/LogReg) — стэкинг с аккуратными OOF-фолдами.

In [ ]:
BEST = None

if len(CANDIDATES) >= 2:
    try:
        run_bl_eq = blend.equal_weight([c["run"] for c in CANDIDATES])
        register_candidate("blend_eq", run_bl_eq)
    except Exception as e:
        print("blend_eq error:", e)

    try:
        scorer = ME.get_scorer(TASK_TYPE, PRIMARY_METRIC)
        ws = blend.weight_search([c["run"] for c in CANDIDATES], y, scorer, nonneg=True, sum_to_one=True)
        print("weight_search:", ws)
    except Exception as e:
        print("weight_search error:", e)

    try:
        run_bl_l2 = blend.ridge_level2([c["run"] for c in CANDIDATES], y, alpha=1.0)
        register_candidate("blend_l2", run_bl_l2)
    except Exception as e:
        print("blend_l2 error:", e)

all_rows = []
for c in CANDIDATES:
    r = c["run"]
    all_rows.append((c["name"], r.cv_mean))
all_rows.sort(key=lambda x: x[1], reverse=True)
print("→ лидер:", all_rows[0] if all_rows else "—")
BEST = next((c for c in CANDIDATES if c["name"]==all_rows[0][0]), None) if all_rows else None


## Калибровка и пороги/Top-K

* Классификация: Platt/Isotonic по OOF, затем глобальный τ (binary) или per-class/top-K.
* Регрессия: калибровка не применяется (оставляем пост-проц в `04_eval_post`).

In [ ]:
CAL = None
TAU = None

if BEST is not None and TASK_TYPE in ("binary","multiclass","multilabel"):
    r = BEST["run"]
    try:
        scorer = ME.get_scorer(TASK_TYPE, PRIMARY_METRIC)
        CAL = calibration.fit(r.oof_true, r.oof_pred, method="platt")
        oof_cal = calibration.apply(CAL, r.oof_pred)

        if TASK_TYPE == "binary":
            TAU = thresholds.find_global_tau(r.oof_true, oof_cal, scorer)
            print("τ:", TAU)
        elif TASK_TYPE == "multilabel":
            if TOP_K is not None:
                print("Используем top-K =", TOP_K)
            else:
                print("Можно искать глобальный τ или per-class τ (осторожно)")
    except Exception as e:
        print("calibration/threshold error:", e)
else:
    print("Calib skipped")

if CAL is not None and BEST is not None:
    run_dir = BEST["run"].artifacts_path
    try:
        MA.save_array(run_dir, "calibrator.json", CAL)
    except Exception:
        try:
            import joblib
            joblib.dump(CAL, run_dir/"calibrator.joblib")
        except Exception:
            pass
    if TAU is not None:
        MA.save_array(run_dir, "thresholds.json", {"tau": float(TAU)})


## Робастность по подгруппам

Сделайте разрезы по времени, гео или крупным категориям — ищите провалы метрики.

In [ ]:
def slice_metric(y_true, y_pred, idx, scorer):
    return scorer(y_true[idx], y_pred[idx])

print("Срезы: добавь при необходимости (гео/время/категории)")


## Финальный инференс на test

Можно использовать уже обученные модели (K-fold инференс из `ModelRun.test_pred`) или
дотренировать на всём трейне. Ниже — аккуратное применение калибровки/порогов и сохранение.

In [ ]:
if BEST is not None:
    r = BEST["run"]
    test_out = r.test_pred

    if CAL is not None:
        test_out = calibration.apply(CAL, test_out)

    post_meta = {}
    if TASK_TYPE == "binary" and TAU is not None:
        yhat = (test_out >= TAU).astype(int)
        post_meta["tau"] = float(TAU)
    elif TASK_TYPE == "multilabel" and TOP_K is not None:
        yhat = thresholds.apply_topk(test_out, TOP_K)
        post_meta["top_k"] = int(TOP_K)
    else:
        yhat = test_out

    run_dir = r.artifacts_path
    MA.save_array(run_dir, "test_post.json", post_meta)
    try:
        np.save(run_dir/"test_post.npy", yhat)
    except Exception:
        pass

    print("Final test artifact →", run_dir)
else:
    print("Нет BEST кандидата")


## Паспорт решения и «что дальше»

* Имя лучшего кандидата, CV mean/std, применялись ли бленд/калибровка/τ.
* Дальше: открыть `04_eval_post.ipynb` для финального пост-проц и подготовки сабмита.

In [ ]:
if BEST:
    r = BEST["run"]
    passport = {
        "best_name": BEST["name"],
        "run_id": str(r.run_id),
        "cv_mean": float(r.cv_mean),
        "cv_std": float(r.cv_std),
        "task": TASK_TYPE,
        "metric": PRIMARY_METRIC,
        "used_models": [c["name"] for c in CANDIDATES],
        "calibration": CAL is not None,
        "tau": None if TAU is None else float(TAU),
        "top_k": TOP_K,
    }
    print(json.dumps(passport, ensure_ascii=False, indent=2))
else:
    print("Паспорт недоступен — нет BEST")


## «Паник-профиль» на последние 40 минут

Если дедлайн совсем близко: обучи **один** GBDT (200–300 деревьев) на `X_dense`
+ **одну** линейку на TF-IDF; сделай equal-weight бленд, Platt и глобальный τ, выдай test.

In [ ]:
def panic_run():
    local_candidates = []
    try:
        params = {**GBDT_PARAMS, "n_estimators": 250, "max_depth": 7}
        run_id = MA.make_run_id(task=TASK_TYPE, model=f"{GBDT_LIB}-panic", feat_hash="panic", seed=SEED)
        run_g = gbdt.train_cv(
            X_dense_tr, y, X_dense_te, FOLDS,
            params=params, run_id=run_id, lib=GBDT_LIB,
            task=TASK_TYPE, seed=SEED, n_jobs=N_JOBS, save=True, resume=True, show_progress=True, verbose=True
        )
        local_candidates.append(run_g)
        print("panic gbdt CV", run_g.cv_mean)
    except Exception as e:
        print("panic gbdt error:", e)

    try:
        run_id = MA.make_run_id(task=TASK_TYPE, model=f"{LIN_ALGO}-panic", feat_hash="panic", seed=SEED)
        run_l = linear.train_cv(
            X_sparse_tr, y, X_sparse_te, FOLDS,
            algo=LIN_ALGO, task=TASK_TYPE, params={**LIN_PARAMS, "max_iter": 150},
            run_id=run_id, seed=SEED, n_jobs=N_JOBS, save=True, resume=True, show_progress=True, verbose=True
        )
        local_candidates.append(run_l)
        print("panic linear CV", run_l.cv_mean)
    except Exception as e:
        print("panic linear error:", e)

    if len(local_candidates) >= 2:
        run_bl = blend.equal_weight(local_candidates)
        print("panic blend CV", run_bl.cv_mean)
    else:
        print("panic: not enough candidates")

# вызвать при необходимости
# panic_run()
